In [49]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import pandas as pd
import os
from tqdm import tqdm
import numpy as np
from torchvision import datasets
from torchvision.transforms import ToTensor
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
from torchvision import datasets
from PIL import Image

In [50]:
train_labels = pd.read_csv("train.csv")
test_labels = pd.read_csv("test.csv")
validation_labels = pd.read_csv("validation.csv")

In [51]:
class CustomImageDataset(Dataset):
    def __init__(self, labels, img_dir, transform=None, target_transform=None):
        self.img_labels = pd.read_csv(labels)
        self.img_dir = img_dir
        self.transform = transform
        self.target_transform = target_transform
    def __len__(self):
        return len(self.img_labels)

    @staticmethod
    def read_image(img_path):
        imagine = np.array(Image.open(img_path))
        imagine = np.ravel(imagine.copy())
        if(imagine.shape == (6400, )):
            imagine = np.concatenate((imagine, imagine, imagine))
        imagine = np.reshape(imagine, (80, 80, 3))
        return np.reshape(imagine, (80, 80, 3))

    def __getitem__(self, idx):
        img_path = os.path.join(self.img_dir, self.img_labels.iloc[idx, 0]) + ".png"
        #print(img_path)
        image = self.read_image(img_path)
        label = self.img_labels.iloc[idx, 1]
        if self.transform:
            image = self.transform(image)
        if self.target_transform:
            label = self.target_transform(label)
        return image, label

In [52]:
train_data = CustomImageDataset("train.csv", "train", transform = ToTensor())
test_data = CustomImageDataset("test.csv", "test", transform = ToTensor())
validation_data = CustomImageDataset("validation.csv", "validation", transform = ToTensor())

In [83]:
class ReteaNeuronala(nn.Module):
    def __init__(self, n_layers, n_neurons):
        super().__init__()
        self.flatten = nn.Flatten()
        self.layers = nn.ModuleList()#e o lista dar se intelege bine cu CUDA
        self.first_layer = nn.Linear(80 * 80 * 3, n_neurons)
        self.layers.append(self.first_layer)
        for _ in range(n_layers):
            self.layers.append(nn.Linear(n_neurons, n_neurons))
        self.output_layer = nn.Linear(n_neurons, 3)

    def forward(self, x):
        x = self.flatten(x)
        for layer in self.layers:
            x = F.relu(layer(x))
        # x = F.relu(self.first_layer(x))
        # x = F.relu(self.second_layer(x))
        x = self.output_layer(x)
        return x        

In [87]:
train_loader = DataLoader(train_data, batch_size = 64)
test_loader = DataLoader(test_data, batch_size = 64)
validation_loader = DataLoader(validation_data, batch_size = 64)

In [88]:
#creem retea si definim algoritmul de optimizare
model = ReteaNeuronala(3, 2352)
device = "cuda" if torch.cuda.is_available() else "cpu"
model = model.to(device)
optimizer = torch.optim.SGD(model.parameters(), lr = 1e-2)

In [89]:
NUM_EPOCHS = 15
print(device)
loss_function = nn.CrossEntropyLoss()

model.train(True)
for i in range(NUM_EPOCHS):
    print(f"=== Epoch{i + 1}===")
    for batch, (image_batch, labels_batch) in enumerate(train_loader):
        image_batch = image_batch.to(device)
        labels_batch = labels_batch.to(device)
        #print(image_batch.shape)
        #print(labels_batch.shape)

        pred = model(image_batch)
        #print(pred.shape)
        loss = loss_function(pred, labels_batch)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        if batch%100 == 0:
            loss = loss.item()
            print(f"Batch index{batch}, loss:{loss:>7f}")


cuda
=== Epoch1===
Batch index0, loss:1.100909
Batch index100, loss:1.096499
=== Epoch2===
Batch index0, loss:1.091160
Batch index100, loss:1.093425
=== Epoch3===
Batch index0, loss:1.082553
Batch index100, loss:1.090652
=== Epoch4===
Batch index0, loss:1.070438
Batch index100, loss:1.089952
=== Epoch5===
Batch index0, loss:1.058501
Batch index100, loss:1.092588
=== Epoch6===
Batch index0, loss:1.051515
Batch index100, loss:1.095601
=== Epoch7===
Batch index0, loss:1.050154
Batch index100, loss:1.097607
=== Epoch8===
Batch index0, loss:1.052832
Batch index100, loss:1.099731
=== Epoch9===
Batch index0, loss:1.057366
Batch index100, loss:1.101680
=== Epoch10===
Batch index0, loss:1.062680
Batch index100, loss:1.104333
=== Epoch11===
Batch index0, loss:1.068392
Batch index100, loss:1.107011
=== Epoch12===
Batch index0, loss:1.074110
Batch index100, loss:1.110250
=== Epoch13===
Batch index0, loss:1.082430
Batch index100, loss:1.115026
=== Epoch14===
Batch index0, loss:1.093111
Batch index1

In [90]:
correct = 0
test_loss = 0
size = len(validation_loader.dataset)
model.to(device)
model.eval()
with torch.no_grad():
    for image_batch, labels_batch in validation_loader:
        image_batch = image_batch.to(device)
        labels_batch = labels_batch.to(device)
        pred = model(image_batch)
        test_loss += loss_function(pred, labels_batch).item()
        correct += (pred.argmax(1) == labels_batch).type(torch.float).sum().item()
correct /= size
test_loss /= size
print(f"Accuracy: {(100*correct):>0.1f}%, Loss: {test_loss:>8f} \n")


Accuracy: 40.3%, Loss: 0.017274 

